# Calculate Power Levels

#### PFD and SPFD to Jy calculations.

Satellite PFD at the surface of the Earth : -146dBW/m2/4kHz

In [20]:
from numpy import log10,pi

## PFD from the SAT (at the Earth's surface)
sat_pfd={'MainLobe':-146,  'SideLobe':-146 - 38.3}  ## dB : W/(m2(4kHz))
## Satellite forward gain = 38.3 dB
##    Power in a sidelobe = Power in a mainlobe (dB) - 38.3 (dB)

## SAT Spectral Power Flux Density (spfd)
sat_spfd={}
for sat_lobe in ['MainLobe','SideLobe']:
    pfd_db = sat_pfd[sat_lobe]
    pfd_w = 10**(pfd_db/10.0)
    spfd_w_hz = pfd_w/4e+3   ## W/(m2 Hz)
    spfd_db_hz = 10*log10(spfd_w_hz)
    print("SPFD incident at the VLA antenna (from SAT %s): %3.2f [dB W/m2/Hz]"%(sat_lobe,spfd_db_hz))
    sat_spfd[sat_lobe]=spfd_db_hz

## PFD from a UT (at a specified distance from the transmitter)
gtx= 0 * cnv.dBi  ## Always a sidelobe
ptx = 3.2 * u.W   ## Transmitted power
ut_dist = 10.0  # km
dist = ut_dist * u.km  ## Distance
rec_pfd = cnv.powerflux_from_ptx(ptx, dist, gtx)   ## W/m2   in a 60MHz transmission band
rec_ut_spfd = rec_pfd /(60e6 * u.Hz)   ## W/m2/Hz
rec_ut_spfd_db = 10*log10(rec_ut_spfd/(1.0*u.W/(u.m*u.m*u.Hz)))
print("SPFD incident at the VLA antenna (from UT SideLobe at %3.2f km distance): %3.2f [dB W/m2/Hz] "%(ut_dist,rec_ut_spfd_db))

rfi_spfd = {'SAT_MainLobe': sat_spfd['MainLobe'], 'SAT_SideLobe':sat_spfd['SideLobe'], 'UT_SideLobe':rec_ut_spfd_db}
rfi_freq = {'SAT_MainLobe': 11e9, 'SAT_SideLobe':11e9, 'UT_SideLobe':14.25e9}


for rfi_lobe in ['SAT_MainLobe','SAT_SideLobe','UT_SideLobe']:
    for vla_lobe in ['MainLobe','SideLobe']:
        
        ## Calculate VLA antenna gain
        if vla_lobe=='SideLobe':
            grx = 0 * cnv.dBi
        else:  # MainLobe ( https://www.everythingrf.com/rf-calculators/parabolic-reflector-antenna-gain)
            d=25.0 # diameter in m
            lam = 3e8/rfi_freq[rfi_lobe] # lamda in m
            grx = 10*log10( 0.75 * (pi*d/lam)**2 )   * cnv.dBi
                    
        spfd_db = rfi_spfd[rfi_lobe]

        freq = rfi_freq[rfi_lobe] * u.Hz
        area = 0.75 * 3.14*(12.5**2)  * u.m*u.m   ## Collecting area of a VLA dish :  m2
        chanbw = 125e3 * u.Hz    ## Chan bw  : Hz

        
        print("\nVLA %s and %s"%(vla_lobe,rfi_lobe))
        print("=============================")

        spfd_w = 10**(spfd_db/10.0) * u.W / (u.m*u.m*u.Hz)  ##  W/m2/Hz
        pfd_rec_chan = spfd_w * chanbw   ## Power Flux Density : W/m2 in one 125kHz channel.... 

        rec_pwr = cnv.prx_from_powerflux(powerflux=pfd_rec_chan, grx=grx, freq=freq)   # W  in a 125kHz channel
        rec_pwr_W = rec_pwr / (1.0*u.W)   # W
        rec_pwr_WdB = 10*log10(rec_pwr_W) # dB W
        print("Recd power in one 125kHz channel :\t %3.2f dB W \t\t[%3.2e W]"%(rec_pwr_WdB, rec_pwr_W))

        rec_spfd = rec_pwr / (area*chanbw)     # W / m2 /Hz    --> to get to Units of Jy...
        rec_spfd_W = rec_spfd / (1.0*u.W/(u.m*u.m*u.Hz))
        rec_spfd_dB = 10*log10(rec_spfd_W)
        print("Recd spectral flux density :\t\t %3.2f dB W/m2/Hz \t[%3.2e W/m2/Hz] [%3.2e Jy]"%(rec_spfd_dB,rec_spfd_W,rec_spfd_W/1e-26))


SPFD incident at the VLA antenna (from SAT MainLobe): -182.02 [dB W/m2/Hz]
SPFD incident at the VLA antenna (from SAT SideLobe): -220.32 [dB W/m2/Hz]
SPFD incident at the VLA antenna (from UT SideLobe at 10.00 km distance): -163.72 [dB W/m2/Hz] 

VLA MainLobe and SAT_MainLobe
Recd power in one 125kHz channel :	 -105.40 dB W 		[2.89e-11 W]
Recd spectral flux density :		 -182.02 dB W/m2/Hz 	[6.27e-19 W/m2/Hz] [6.27e+07 Jy]

VLA SideLobe and SAT_MainLobe
Recd power in one 125kHz channel :	 -173.34 dB W 		[4.64e-18 W]
Recd spectral flux density :		 -249.96 dB W/m2/Hz 	[1.01e-25 W/m2/Hz] [1.01e+01 Jy]

VLA MainLobe and SAT_SideLobe
Recd power in one 125kHz channel :	 -143.70 dB W 		[4.27e-15 W]
Recd spectral flux density :		 -220.32 dB W/m2/Hz 	[9.28e-23 W/m2/Hz] [9.28e+03 Jy]

VLA SideLobe and SAT_SideLobe
Recd power in one 125kHz channel :	 -211.64 dB W 		[6.86e-22 W]
Recd spectral flux density :		 -288.26 dB W/m2/Hz 	[1.49e-29 W/m2/Hz] [1.49e-03 Jy]

VLA MainLobe and UT_SideLobe
Recd pow